# MNIST

Данные с сайта https://archive.ics.uci.edu/ml/datasets/Cervical+cancer+%28Risk+Factors%29#

In [74]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [75]:
train_loader = torch.utils.data.DataLoader(datasets.MNIST('../mnist_data', 
                                                          download=True, 
                                                          train=True,
                                                          transform=transforms.Compose([
                                                              transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                              transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
                                                          ])), 
                                           batch_size=10, shuffle=True)

Processing...
Done!


In [76]:
test_loader = torch.utils.data.DataLoader(datasets.MNIST('../mnist_data', 
                                                          download=True, 
                                                          train=False,
                                                          transform=transforms.Compose([
                                                              transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                              transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
                                                          ])), 
                                           batch_size=10, shuffle=True)

In [78]:
class CNNClassifier(nn.Module):
    def __init__(self):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.dropout = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.dropout(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        
        return F.log_softmax(x)

In [97]:
clf = CNNClassifier()
opt = optim.SGD(clf.parameters(), lr=0.01, momentum=0.5)

In [80]:
loss_history = []
acc_history = []

def train(epoch):
    clf.train() # set model in training mode (need this because of dropout)
    
    # dataset API gives us pythonic batching 
    for batch_id, (data, label) in enumerate(train_loader.):
        data = Variable(data)
        target = Variable(label)
        
        # forward pass, calculate loss and backprop!
        opt.zero_grad()
        preds = clf(data)
        loss = F.nll_loss(preds, target)
        loss.backward()
        loss_history.append(loss.data[0])
        opt.step()
        
        if batch_id % 100 == 0:
            print(loss.data[0])

def test(epoch):
    clf.eval() # set model in inference mode (need this because of dropout)
    test_loss = 0
    correct = 0
    
    for data, target in test_loader:
        data = Variable(data, volatile=True) 
        target = Variable(target)
        
        output = clf(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss = test_loss
    test_loss /= len(test_loader) # loss function already averages over batch size
    accuracy = 100. * correct / len(test_loader.dataset)
    acc_history.append(accuracy)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        accuracy))

In [81]:
for epoch in range(0, 3):
    print("Epoch %d" % epoch)
    train(epoch)
test(epoch)

Epoch 0
tensor(2.3801)


C:\Users\morgachev\Miniconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\morgachev\Miniconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
C:\Users\morgachev\Miniconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


tensor(2.0333)
tensor(1.4325)
tensor(0.5938)
tensor(0.9558)
tensor(0.5642)
tensor(0.7700)
tensor(0.6083)
tensor(0.4941)
tensor(0.4757)
tensor(0.2336)
tensor(0.1844)
tensor(1.5739)
tensor(0.6354)
tensor(0.4957)
tensor(0.1441)
tensor(0.5711)
tensor(0.6856)
tensor(0.9733)
tensor(0.4700)
tensor(0.7748)
tensor(0.2523)
tensor(0.2375)
tensor(0.5631)
tensor(0.3589)
tensor(0.3372)
tensor(0.4123)
tensor(0.4687)
tensor(0.2272)
tensor(0.5105)
tensor(0.2588)
tensor(0.1793)
tensor(0.1335)
tensor(0.2776)
tensor(0.3351)
tensor(0.0876)
tensor(0.5009)
tensor(0.5970)
tensor(0.3390)
tensor(0.1769)
tensor(0.1534)
tensor(0.2362)
tensor(0.0481)
tensor(0.1082)
tensor(0.0171)
tensor(0.3210)
tensor(0.6217)
tensor(0.6603)
tensor(0.4820)
tensor(0.0318)
tensor(0.3331)
tensor(0.6362)
tensor(0.2913)
tensor(0.2771)
tensor(0.1733)
tensor(0.0368)
tensor(0.1000)
tensor(0.8321)
tensor(0.0853)
tensor(0.1202)
Epoch 1
tensor(0.0429)
tensor(0.1585)
tensor(0.2543)
tensor(0.0087)
tensor(0.3770)
tensor(0.1763)
tensor(0.9133)
te

C:\Users\morgachev\Miniconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
C:\Users\morgachev\Miniconda3\lib\site-packages\ipykernel_launcher.py:33: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number



Test set: Average loss: 0.0527, Accuracy: 9842/10000 (98%)

